<h4> Save for Storyboard </h4>

In [ ]:
import struct, socket
import numpy as np 
import csv, json 
import os 
import urllib2 
import datetime
import operator

#import ipywidgets as widgets # if your using jupyter/ipython < 1.4 please comment this line and uncomment the lines below
from IPython.html import widgets
from IPython.html.widgets import interact, interactive, fixed
from IPython.display import display, HTML, clear_output, Javascript 


path = os.getcwd().split("/") 
date = path[len(path)-1] 
spath = os.getcwd() 
data_src = 'http://localhost:8895/app/api/dns/threat_investigation/' 
content_type = {'Content-Type':'application/json'}
anchor = ''
anchor_type = '' 

**Functions Definition**

In [ ]:
###Searches for the DNS scored as Severe (sev = 1)
def start_investigation():
    display_expanded_search()

def display_expanded_search():
    ips_query = {}  
    url = data_src + date
    reader = json.loads(urllib2.urlopen(url).read())
       
    for row in reader['data']:    
        if row['ip_sev'] == 1: 
            ips_query[row['ip_dst']]='i'
        if row['dns_sev'] == 1:
            ips_query[row['ip_dst']]='q' 
            
    if len(ips_query) == 0:
        print "There are not high risk results."
    else:  
        sorted_dict = sorted(ips_query.items(), key=operator.itemgetter(0))      
        display_controls(sorted_dict)   
        
        
def display_controls(ip_list): 
    cont_list = widgets.HBox(width=400, height=220)    
    cont_h_list = widgets.Box(width=400, height=250)    
    label = widgets.HTML(value='<h4>Expanded Search</h4>')
    ip_select = widgets.Select(options=ip_list,height=min((len(ip_list)*18),200),width=250)
    
    search_ip_button = widgets.Button(description='Search')
    cont_list.children = [ip_select,search_ip_button]
    cont_h_list.children = [label,cont_list] 
    
    display(cont_h_list)
  
    def search_ip(b): 
        global anchor 
        global anchor_type
        anchor = ''
        anchor_type = ''
        anchor = ip_select.selected_label  
        anchor_type = ip_select.value         
        table = "<table><th>IP</th><th>QUERY</th><th>TOTAL</th>"
         
        if anchor_type == 'i':
            params  = '{"ip_dst":"'+ anchor + '"}'
        elif anchor_type == 'q':
            params  = '{"ip_dst":"'+ anchor + '"}'
            
        url = data_src + "visual/" + date 
        request = urllib2.Request(url, params, content_type)
        results = json.loads(urllib2.urlopen(request).read()) 
          
        if len(results['data']) > 0:
            for row in results['data']:  
                table += "<tr><td>" +  row['ip_dst'] + "</td><td>" + row['dns_qry_name'] + "</td><td>" + str(row['total']) + "</td></tr>"  
            table += "</table>"
            clear_output()   
            display(HTML(table)) 
        else:
            clear_output()   
            print "No additional threats found for {0}".format(anchor) 
        display_threat_box(anchor)
    
    search_ip_button.on_click(search_ip)

        
def display_threat_box(ip):   
    tc_box_main = widgets.Box(width=550, height=200)
    
    threat_container = widgets.HBox(width=550, height=150)
    tc_box_separator = widgets.Box(width=550, height=15)     
    tc_div_label = widgets.HTML(value='<h4>THREAT SUMMARY FOR ' + anchor +'</h4>')  
          
    tc_txt_title = widgets.Text(value='',width=200, placeholder='Threat Title')
    tc_txa_summary = widgets.Textarea(value=''
                                      ,width=300, height=100)
    tc_btn_save = widgets.Button(description='Save')
    threat_container.children = [tc_txa_summary,tc_btn_save]
    tc_box_main.children = [tc_div_label, tc_txt_title, tc_box_separator, threat_container] 
    removeWidget(1) 
    display(tc_box_main) 
    
    def save_threat_summary(b):
        global anchor
        anchor_ip =''
        anchor_dns ='' 
        if anchor != '':
            print "Loading ..."
            file_name = spath + "/threat_comments.csv" 
            if anchor_type == 'i':
                anchor_ip = anchor
            elif anchor_type == 'q':
                anchor_dns = anchor
                
            dt = str(datetime.datetime.now())
            yy = date[0:4]
            mm = date[4:6]
            dd = date[6:8]
            usr = "IPython"
    
            file_content = anchor_ip + '|' + anchor_dns + '|' + tc_txt_title.value + '|' + tc_txa_summary.value.replace('\n', '\\n') + '|' + dt + '|' + usr + '|' + yy + '|' + mm + '|' + dd + "\n"
            
            with open(file_name, 'a') as comment:
                comment.write(file_content)                 
            url = data_src + "comments/" + date  
            params  = '{"data":"' + file_name + '"}'  
            results = urllib2.urlopen(urllib2.Request(url, params, content_type)).read()
            os.remove(file_name)    
            
            if results:
                clear_output()
                tc_box_main.close() 
                display(Javascript("$(\"option[data-value='" + anchor  +"']\").remove();"))        
                anchor = ''
                print "Comments saved"
            else:
                print results
            
    tc_btn_save.on_click(save_threat_summary)


def removeWidget(index):
    js_command = "var widget_list = $('.widget-area').children('.widget-subarea').children('.widget-box'); \n"
    js_command = js_command + "if(widget_list.length > {0} ) \n ".format(index)
    js_command = js_command + "widget_list[{0}].remove(); \n ".format(index)
    
    display(Javascript(js_command))   

In [ ]:
start_investigation()